<a href="https://colab.research.google.com/github/aranjaka/dw_matrix_car/blob/master/day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [0]:
!pwd

/content


In [0]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [0]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

# Feature engineering

In [0]:
SUFFIX_CAT = '_cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [0]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19747.810938471786, 75.95212671816479)

# RandomForest

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18864.27843495869, 38.24233578175403)

# XGBoost

In [0]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}


run_model(xgb.XGBRegressor(**xgb_params), cat_feats )

[20:37:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:37:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:37:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13034.249470063352, 103.73566521784606)

In [0]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

[20:41:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1274 ± 0.0017,param_napęd_cat
0.1159 ± 0.0025,param_stan_cat
0.1131 ± 0.0037,param_rok-produkcji_cat
0.0565 ± 0.0010,param_faktura-vat_cat
0.0482 ± 0.0014,param_moc_cat
0.0416 ± 0.0017,param_skrzynia-biegów_cat
0.0252 ± 0.0010,param_marka-pojazdu_cat
0.0247 ± 0.0014,feature_kamera-cofania_cat
0.0232 ± 0.0008,param_typ_cat
0.0198 ± 0.0006,param_pojemność-skokowa_cat


In [0]:
feats = ['param_napęd_cat','param_stan_cat','param_rok-produkcji_cat','param_faktura-vat_cat','param_moc_cat','param_skrzynia-biegów_cat','param_marka-pojazdu_cat','feature_kamera-cofania_cat','param_typ_cat','param_pojemność-skokowa_cat','seller_name_cat','param_wersja_cat','feature_wspomaganie-kierownicy_cat','param_model-pojazdu_cat','feature_system-start-stop_cat','param_kod-silnika_cat','feature_asystent-pasa-ruchu_cat','feature_łopatki-zmiany-biegów_cat','feature_światła-led_cat','feature_czujniki-parkowania-przednie_cat']
run_model(xgb.XGBRegressor(**xgb_params), feats )

[20:53:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:53:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:53:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13407.229151187508, 47.968383519686824)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x) )
feats = ['param_napęd_cat','param_stan_cat','param_rok-produkcji','param_faktura-vat_cat','param_moc_cat','param_skrzynia-biegów_cat','param_marka-pojazdu_cat','feature_kamera-cofania_cat','param_typ_cat','param_pojemność-skokowa_cat','seller_name_cat','param_wersja_cat','feature_wspomaganie-kierownicy_cat','param_model-pojazdu_cat','feature_system-start-stop_cat','param_kod-silnika_cat','feature_asystent-pasa-ruchu_cat','feature_łopatki-zmiany-biegów_cat','feature_światła-led_cat','feature_czujniki-parkowania-przednie_cat']
run_model(xgb.XGBRegressor(**xgb_params), feats )

[21:36:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:36:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:36:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11364.016578827623, 80.05911838700821)

In [0]:
df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).replace(' ', '').replace('KM','')) )

0          90
2         115
3         262
4         110
5         310
         ... 
160609     75
160610     75
160611    120
160614    150
160615     88
Name: param_moc, Length: 106494, dtype: int64

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x) )
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).replace(' ', '').replace('KM','')) )
feats = ['param_napęd_cat','param_stan_cat','param_rok-produkcji','param_faktura-vat_cat','param_moc','param_skrzynia-biegów_cat','param_marka-pojazdu_cat','feature_kamera-cofania_cat','param_typ_cat','param_pojemność-skokowa_cat','seller_name_cat','param_wersja_cat','feature_wspomaganie-kierownicy_cat','param_model-pojazdu_cat','feature_system-start-stop_cat','param_kod-silnika_cat','feature_asystent-pasa-ruchu_cat','feature_łopatki-zmiany-biegów_cat','feature_światła-led_cat','feature_czujniki-parkowania-przednie_cat']
run_model(xgb.XGBRegressor(**xgb_params), feats )

[21:42:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:42:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:42:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9724.786775066808, 84.17027602984261)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).replace(' ', '').replace('cm3','')) )
feats = ['param_napęd_cat','param_stan_cat','param_rok-produkcji','param_faktura-vat_cat','param_moc','param_skrzynia-biegów_cat','param_marka-pojazdu_cat','feature_kamera-cofania_cat','param_typ_cat','param_pojemność-skokowa','seller_name_cat','param_wersja_cat','feature_wspomaganie-kierownicy_cat','param_model-pojazdu_cat','feature_system-start-stop_cat','param_kod-silnika_cat','feature_asystent-pasa-ruchu_cat','feature_łopatki-zmiany-biegów_cat','feature_światła-led_cat','feature_czujniki-parkowania-przednie_cat']
run_model(xgb.XGBRegressor(**xgb_params), feats )

[21:47:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:47:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:47:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9570.7287270162, 87.36868463251476)

In [0]:
ls

data/                     day3_simple_model.ipynb  LICENSE
day2_visualisation.ipynb  day4.ipynb               README.md
